# Two layer net

## 背景

为了提高预测准确率，我们需要使用多层神经网络，因为一般来说网络的层数越多其表达能力越强，因为其参数更多，能表达出的状态信息更多，所以表达能力越强。多层神经网络可以应对更加复杂的问题，这一节我们先从一个小例子开始：构建一个两层神经网络，这一节我们构建的简单两层神经网络可以达到51%的准确率。

## 构建神经网络

In [1]:
import $plugin.$ivy.`com.thoughtworks.implicit-dependent-type::implicit-dependent-type:2.0.0`

import $ivy.`com.thoughtworks.deeplearning::differentiableany:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablenothing:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiableseq:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiabledouble:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablefloat:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablehlist:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiablecoproduct:1.0.0-RC7`
import $ivy.`com.thoughtworks.deeplearning::differentiableindarray:1.0.0-RC7`
import $ivy.`org.rauschig:jarchivelib:0.5.0`

import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import java.io.{FileInputStream, InputStream}


import com.thoughtworks.deeplearning
import org.nd4j.linalg.api.ndarray.INDArray
import com.thoughtworks.deeplearning.DifferentiableHList._
import com.thoughtworks.deeplearning.DifferentiableDouble._
import com.thoughtworks.deeplearning.DifferentiableINDArray._
import com.thoughtworks.deeplearning.DifferentiableAny._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Optimizers._
import com.thoughtworks.deeplearning.DifferentiableINDArray.Layers.Weight
import com.thoughtworks.deeplearning.{
  DifferentiableHList,
  DifferentiableINDArray,
  Layer,
  Symbolic
}
import com.thoughtworks.deeplearning.Layer.Batch
import com.thoughtworks.deeplearning.Symbolic.Layers.Identity
import com.thoughtworks.deeplearning.Symbolic._
import com.thoughtworks.deeplearning.Poly.MathFunctions._
import com.thoughtworks.deeplearning.Poly.MathMethods./
import com.thoughtworks.deeplearning.Poly.MathOps
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.cpu.nativecpu.NDArray
import org.nd4j.linalg.factory.Nd4j
import org.nd4j.linalg.indexing.{INDArrayIndex, NDArrayIndex}
import org.nd4j.linalg.ops.transforms.Transforms
import org.nd4s.Implicits._
import shapeless._

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

import scala.collection.immutable.IndexedSeq
import scala.util.Random

pprintConfig() = pprintConfig().copy(height = 5)//减少输出的行数，避免页面输出太长

import $file.ReadCIFAR10ToNDArray,ReadCIFAR10ToNDArray._
import $file.Utils,Utils._

import $plugin.$                                                                             


import $ivy.$                                                           

import $ivy.$                                                               

import $ivy.$                                                           

import $ivy.$                                                              

import $ivy.$                                                             

import $ivy.$                                                             

import $ivy.$                                                                 

import $ivy.$                                                                

import $ivy.$                               


import $ivy.$                                             


import java.io.{FileInputStream, InputStream}



import com.thoughtworks.deeplearning

import org.nd4j.linalg.api.ndarray.INDArray

import com.thoughtworks.deeplearning.Differentiabl

类似[前一节](https://thoughtworksinc.github.io/DeepLearning.scala/demo/MiniBatchGradientDescent.html)从CIFAR10 database中读取和处理测试数据的图片和标签信息

In [2]:
//CIFAR10中的图片共有10个分类(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val NumberOfClasses: Int = 10

//加载测试数据，我们读取100条作为测试数据
val testNDArray =
  ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

val testData = testNDArray.head

val testExpectResult = testNDArray.tail.head

val vectorizedTestExpectResult = Utils.makeVectorized(testExpectResult, NumberOfClasses)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


NumberOfClasses: Int = 10
testNDArray: INDArray :: INDArray :: HNil = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0.55, 0.55, 0.56, 0.54, 0.49, 0.45, 0.59, 0.59, 0.62, 0.65, 0.63, 0.62, 0.64, 0.63, 0.64, 0.61, 0.61, 0.62, 0.64, 0.66, 0.67, 0.67, 0.66, 0.62, 0.60, 0.59, 0.57, 0.54, 0.55, 0.55, 0.58, 0.57, 0.57, 0.55, 0.56, 0.53, 0.49, 0.46, 0.59, 0.59, 0....
testData: INDArray = [[0.62, 0.62, 0.64, 0.65, 0.62, 0.61, 0.63, 0.62, 0.62, 0.62, 0.63, 0.62, 0.63, 0.65, 0.66, 0.66, 0.65, 0.63, 0.62, 0.62, 0.61, 0.58, 0.59, 0.58, 0.58, 0.56, 0.55, 0.55, 0.56, 0.54, 0.49, 0.45, 0.59, 0.59, 0.62, 0.65, 0.63, 0.62, 0.64, 0.63, 0.64, 0.61, 0.61, 0.62, 0.64, 0.66, 0.67, 0.67, 0.66, 0.62, 0.60, 0.59, 0.57, 0.54, 0.55, 0.55, 0.58, 0.57, 0.57, 0.55, 0.56, 0.53, 0.49, 0.46, 0.59, 0.59, 0....
testExpectResult: INDArray = [3.00, 8.00, 8.00, 0.00, 6.00, 6.00, 1.00, 6.00, 3.00, 1.00, 0.00, 9.00,

跟前一节不同，这节我们假如一些参数调优的手段，设置学习率和使用[L2Regularization](https://en.wikipedia.org/wiki/Regularization_(mathematics)),L2Regularization可以用来避免过拟合。我们还使用了每个迭代learningRate都下降为原来的0.9995倍的办法来解决训练时间增长时因为learningRate相对太大导致loss下降太慢或者不下降的问题。

In [3]:
implicit val optimizerFactory = new DifferentiableINDArray.OptimizerFactory {
  override def ndArrayOptimizer(weight: Weight): Optimizer = {
    new LearningRate with L2Regularization {

      var learningRate = 0.001

      override protected def currentLearningRate(): Double = {
        learningRate *= 0.9995
        learningRate
      }

      override protected def l2Regularization: Double = 0.03
    }
  }
}

optimizerFactory: AnyRef with OptimizerFactory = $sess.cmd2Wrapper$Helper$$anon$2@79671b4a

编写两层神经网络的第一层神经网络，这是一层全连接一层[relu](https://en.wikipedia.org/wiki/Rectifier_(neural_networks))的神经网络。

In [4]:
def fullyConnectedThenRelu(inputSize: Int, outputSize: Int)(
    implicit row: INDArray @Symbolic): INDArray @Symbolic = {
  val w = (Nd4j.randn(inputSize, outputSize) / math.sqrt(outputSize / 2.0)).toWeight * 0.1
  val b = Nd4j.zeros(outputSize).toWeight
  max((row dot w) + b, 0.0)
}

defined function fullyConnectedThenRelu

跟上节相同，我们使用softmax作为分类器

In [5]:
def softmax(implicit scores: INDArray @Symbolic): INDArray @Symbolic = {
  val expScores = exp(scores)
  expScores / expScores.sum(1)
}

defined function softmax

编写两层神经网络的第二层神经网络，这时一层全连接一层softmax的神经网络。

In [6]:
def fullyConnectedThenSoftmax(inputSize: Int, outputSize: Int)(
    implicit row: INDArray @Symbolic): INDArray @Symbolic = {
  val w = (Nd4j.randn(inputSize, outputSize) / math.sqrt(outputSize)).toWeight //* 0.1
  val b = Nd4j.zeros(outputSize).toWeight
  softmax.compose((row dot w) + b)
}

defined function fullyConnectedThenSoftmax

为了实现两层神经网络我们将上面的两层神经网络组合起来，组成一个两层神经网络。

In [7]:
def hiddenLayer(implicit input: INDArray @Symbolic): INDArray @Symbolic = {
  val layer0 = fullyConnectedThenRelu(3072, 500).compose(input)
  fullyConnectedThenSoftmax(500, 10).compose(layer0)
}

val predictor = hiddenLayer

defined function hiddenLayer
predictor: (Symbolic.To[INDArray]{type OutputData = org.nd4j.linalg.api.ndarray.INDArray;type OutputDelta = org.nd4j.linalg.api.ndarray.INDArray;type InputData = org.nd4j.linalg.api.ndarray.INDArray;type InputDelta = org.nd4j.linalg.api.ndarray.INDArray})#@ = Compose(Compose(MultiplyINDArray(Exp(Identity()),Reciprocal(Sum(Exp(Identity()),WrappedArray(1)))),PlusINDArray(Dot(Identity(),Weight([[0.48, -0.53, 0.27, -0.03, 0.86, -0.18, -0.08, 0.11, 0.50, 0.30],
 [-0.34, 0.27, 0.38, 0.19, -0.39, -0.33, 0.24, 1.10, -0.33, 0.02],
 [0.28, 0.23, -0.47, 0.31, -0.25, -0.16, 0.36, -0.49, -0.33, -0.21],
...

跟上节类似，编写损失函数Loss Function并组合输入层和隐含层

In [8]:
def crossEntropy(
    implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val score = pair.head
  val label = pair.tail.head
  -(label * log(score * 0.9 + 0.1) + (1.0 - label) * log(1.0 - score * 0.9)).mean
}

def network(
   implicit pair: (INDArray :: INDArray :: HNil) @Symbolic): Double @Symbolic = {
  val input = pair.head
  val label = pair.tail.head
  val score: INDArray @Symbolic = predictor.compose(input)
  val hnilLayer: HNil @Symbolic = HNil
  crossEntropy.compose(score :: label :: hnilLayer)
}

val trainer = network

defined function crossEntropy
defined function network
trainer: (Symbolic.To[Double]{type OutputData = Double;type OutputDelta = Double;type InputData = shapeless.::[org.nd4j.linalg.api.ndarray.INDArray,shapeless.::[org.nd4j.linalg.api.ndarray.INDArray,shapeless.HNil]];type InputDelta = shapeless.:+:[org.nd4j.linalg.api.ndarray.INDArray,shapeless.:+:[org.nd4j.linalg.api.ndarray.INDArray,shapeless.CNil]]})#@ = Compose(Negative(ReduceMean(PlusINDArray(MultiplyINDArray(Head(Tail(Identity())),Log(PlusDouble(MultiplyDouble(Head(Identity()),Literal(0.9)),Literal(0.1)))),MultiplyINDArray(PlusDouble(Negative(Head(Tail(Identity()))),Literal(1.0)),Log(PlusDouble(Negative(MultiplyDouble(Head(Identity()),Literal(0.9))),Literal(1.0))))))),HCons(Compose(Compose(Compose(MultiplyINDArray(Exp(Identity()),Reciprocal(Sum...

与上一节相同，训练神经网络并观察每次训练loss的变化

In [9]:
val random = new Random

val MiniBatchSize = 256

val lossSeq =
  (
    for (_ <- 0 to 50) yield {
      val randomIndex = random
        .shuffle[Int, IndexedSeq](0 until 10000) //https://issues.scala-lang.org/browse/SI-6948
        .toArray
      for (times <- 0 until 10000 / MiniBatchSize) yield {
        val randomIndexArray =
          randomIndex.slice(times * MiniBatchSize,
                            (times + 1) * MiniBatchSize)
        val trainNDArray :: expectLabel :: shapeless.HNil =
          ReadCIFAR10ToNDArray.getSGDTrainNDArray(randomIndexArray)
        val input =
          trainNDArray.reshape(MiniBatchSize, 3072)

        val expectLabelVectorized =
          Utils.makeVectorized(expectLabel, NumberOfClasses)
        val loss = trainer.train(input :: expectLabelVectorized :: HNil)
        if(times == 0){
          println(loss)
        }
        loss
      }
    }
  ).flatten

val plot = Seq(
  Scatter(lossSeq.indices, lossSeq)
)

plot.plot(
  title = "loss by time"
)

0.2686176776885986
0.24069557189941407
0.2286484956741333
0.22166686058044432
0.2212003707885742
0.2047487735748291
0.2205038547515869
0.2037975788116455
0.2008286476135254
0.20051753520965576
0.1977792501449585
0.19189105033874512
0.19221527576446534
0.19379620552062987
0.18764128684997558
0.19361141920089722
0.20302889347076417
0.18837952613830566
0.1835470676422119
0.19125791788101196
0.18389480113983153
0.19737205505371094
0.19984123706817628
0.1866359829902649
0.18605028390884398
0.18975214958190917
0.1769115447998047
0.18534992933273314
0.18796164989471437
0.18478775024414062
0.18617348670959472
0.18978433609008788
0.18722929954528808
0.18470964431762696
0.17999396324157715
0.17767066955566407
0.17470579147338866
0.18521106243133545
0.18271849155426026
0.18173846006393432
0.17681776285171508
0.18871110677719116
0.1836523413658142
0.18117042779922485
0.17274922132492065
0.18421320915222167
0.1865368366241455
0.17430260181427001
0.1857973098754883
0.17410311698913575
0.179014551639

random: Random = scala.util.Random@7549cad6
MiniBatchSize: Int = 256
lossSeq: IndexedSeq[Symbolic.To.<refinement>.this.type.OutputData] = Vector(
  0.2686176776885986,
  0.2694873332977295,
  0.27004404067993165,
  0.2747493743896484,
...
plot: Seq[Scatter] = List(
  Scatter(
    Some(
      Doubles(
        Vector(
...
res8_4: String = "plot-1370164548"

跟上一节相同，我们使用测试数据来查看神经网络判断结果并计算准确率。这次准确率应该会上升到51%左右。

In [10]:
val result = predictor.predict(testData)
println(s"result: $result") //输出判断结果

val right = Utils.getAccuracy(result, testExpectResult)
println(s"the result is $right %")

result: [[0.01, 0.15, 0.11, 0.40, 0.01, 0.13, 0.12, 0.00, 0.06, 0.00],
 [0.06, 0.09, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.43, 0.41],
 [0.17, 0.06, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 0.06],
 [0.37, 0.04, 0.03, 0.01, 0.01, 0.00, 0.00, 0.01, 0.52, 0.01],
 [0.00, 0.01, 0.11, 0.04, 0.62, 0.03, 0.15, 0.03, 0.01, 0.00],
 [0.00, 0.06, 0.02, 0.18, 0.02, 0.09, 0.60, 0.01, 0.00, 0.02],
 [0.00, 0.11, 0.01, 0.55, 0.00, 0.23, 0.10, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.19, 0.03, 0.34, 0.01, 0.38, 0.05, 0.00, 0.00],
 [0.01, 0.04, 0.30, 0.18, 0.16, 0.20, 0.03, 0.03, 0.02, 0.00],
 [0.02, 0.77, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.15, 0.05],
 [0.21, 0.01, 0.00, 0.01, 0.00, 0.00, 0.00, 0.00, 0.76, 0.01],
 [0.00, 0.22, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.02, 0.75],
 [0.00, 0.26, 0.02, 0.24, 0.05, 0.12, 0.27, 0.04, 0.01, 0.01],
 [0.05, 0.44, 0.01, 0.08, 0.00, 0.02, 0.24, 0.06, 0.02, 0.06],
 [0.05, 0.38, 0.06, 0.03, 0.01, 0.00, 0.02, 0.06, 0.17, 0.21],
 [0.12, 0.00, 0.14, 0.03, 0.17, 0.03, 0.02, 0.0

result: Symbolic.To.<refinement>.this.type.OutputData = [[0.01, 0.15, 0.11, 0.40, 0.01, 0.13, 0.12, 0.00, 0.06, 0.00],
 [0.06, 0.09, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.43, 0.41],
 [0.17, 0.06, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.70, 0.06],
 [0.37, 0.04, 0.03, 0.01, 0.01, 0.00, 0.00, 0.01, 0.52, 0.01],
 [0.00, 0.01, 0.11, 0.04, 0.62, 0.03, 0.15, 0.03, 0.01, 0.00],
...
right: Double = 51.0


[完整代码](https://github.com/izhangzhihao/deeplearning-tutorial/blob/master/src/main/scala/com/thoughtworks/deeplearning/tutorial/TwoLayerNet.scala)